In [96]:
import numpy as np

In [97]:
def rbm_phi(s, a, b, W):
    c = np.dot(a, s)
    d = 1
    for i in range(len(b)):
        d *= np.cosh(b[i] + np.dot(W[i], s))
        

    return np.exp(c) * d


def rbm_phi_flat(s, a, b, W):
    W = np.reshape(W, (len(b), len(s)))
    return rbm_phi(s, a, b, W)

In [98]:
def o_der_a(s, a, b, W):
    return s

In [99]:
def o_der_b(s, a, b, W):
    return np.array([np.tanh(b[i] + np.dot(W[i], s)) for i in range(len(b))])

In [100]:
def o_der_W(s, a, b, W):
    return np.array([[s[k] * np.tanh(b[i] + np.dot(W[i], s)) for k in range(len(W[0]))] for i in range(len(W))])


In [101]:
# Let's text the derivative functions by comparing them to finite difference approximations.

def finite_diff(f, x, i, h):
    return (f(x + h * np.eye(len(x))[i]) - f(x - h * np.eye(len(x))[i])) / (2 * h)

def finite_diff_grad(f, x, h):
    return np.array([finite_diff(f, x, i, h) for i in range(len(x))])

def test_derivatives():
    a = np.array([0.1, 0.2])
    b = np.array([0.3, 0.4])
    W = np.array([[0.5, 0.6], [0.7, 0.8]])
    s = np.array([1, 0])


    print("RBM values:")
    print(rbm_phi(s, a, b, W))


    h = 1e-6
    print("Analytical derivatives:")
    print(o_der_a(s, a, b, W)*rbm_phi(s, a, b, W))
    print(o_der_b(s, a, b, W)*rbm_phi(s, a, b, W))
    print(o_der_W(s, a, b, W)*rbm_phi(s, a, b, W).flatten())

    print("Finite difference derivatives:")
    print(finite_diff_grad(lambda x: rbm_phi(s, x, b, W), a, h))
    print(finite_diff_grad(lambda x: rbm_phi(s, a, x, W), b, h))
    print(finite_diff_grad(lambda x: rbm_phi_flat(s, a, b, x), W.flatten(), h))



In [102]:
test_derivatives()

RBM values:
2.466227609467873
Analytical derivatives:
[2.46622761 0.        ]
[1.63766582 1.97421279]
[[1.63766582 0.        ]
 [1.97421279 0.        ]]
Finite difference derivatives:
[2.46622761 0.        ]
[1.63766582 1.97421279]
[1.63766582 0.         1.97421279 0.        ]


In [103]:
def rbm_phi_theta(s, theta):
    n = len(s)
    m = (len(theta) - n) // (n+1)
    a = theta[:n]
    b = theta[n:n+m]
    W = np.reshape(theta[n+m:], (m, n))
    return rbm_phi(s, a, b, W)

def o_der_theta(s, theta):
    n = len(s)
    m = (len(theta) - n) // (n+1)
    a = theta[:n]
    b = theta[n:n+m]
    W = np.reshape(theta[n+m:], (m, n))
    return np.concatenate([o_der_a(s, a, b, W), o_der_b(s, a, b, W), o_der_W(s, a, b, W).flatten()])

def grad_rbm_phi_theta(s, theta):
    return o_der_theta(s, theta) * rbm_phi_theta(s, theta)




In [104]:
def create_random_state(N):

    phi = 2*np.random.rand(2**N)-1 + 1j * (2*np.random.rand(2**N)-1)
    #phi = phi/2
    #normalize phi^2
    phi = phi / np.sqrt(np.vdot(phi, phi))
    return phi



In [105]:
#N,M = 4,2
#theta = 0.01*(2*np.random.rand(N + M*(N+1))-1)
#s = [1,1,1,1]
#print(grad_rbm_phi_theta(s, theta))
#print(finite_diff_grad(lambda x: rbm_phi_theta(s, x),theta,1e-6))




In [106]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def summed_powerset(iterable):
    a = [sum(x) for x in powerset(iterable)]
    dim = len(a[1]) if len(a) > 1 else 0
    a[0] = np.zeros(dim)
    return a

def all_states(N):
    return summed_powerset([np.eye(N)[i] for i in range(N)])

print(all_states(4))

[array([0., 0., 0., 0.]), array([1., 0., 0., 0.]), array([0., 1., 0., 0.]), array([0., 0., 1., 0.]), array([0., 0., 0., 1.]), array([1., 1., 0., 0.]), array([1., 0., 1., 0.]), array([1., 0., 0., 1.]), array([0., 1., 1., 0.]), array([0., 1., 0., 1.]), array([0., 0., 1., 1.]), array([1., 1., 1., 0.]), array([1., 1., 0., 1.]), array([1., 0., 1., 1.]), array([0., 1., 1., 1.]), array([1., 1., 1., 1.])]


In [107]:
def loss(phi, theta):
    # N is log2 of the number of states
    N = int(np.log2(len(phi)))
    l = 0
    general_state = all_states(N)
    length = len(general_state)
    magnitude = np.linalg.norm([rbm_phi_theta(general_state[j], theta) for j in range(length)])
    for i in range(length):
        si = general_state[i]
        l += np.vdot(phi[i] - rbm_phi_theta(si, theta)/magnitude, phi[i] - rbm_phi_theta(si, theta)/magnitude)
    return l.real/2

In [108]:
def diff_loss(phi, theta):
    N = int(np.log2(len(phi)))
    general_state = all_states(N)
    length = len(general_state)
    magnitude = np.linalg.norm([rbm_phi_theta(general_state[j], theta) for j in range(length)])
    d = 0
    for i in range(length):
        si = general_state[i]
        summedgradtimesphi = sum([grad_rbm_phi_theta(general_state[j], theta)*rbm_phi_theta(general_state[j], theta) for j in range(length)])
        deriv = (phi[i] - rbm_phi_theta(si, theta)/magnitude) * -(grad_rbm_phi_theta(si, theta)/magnitude - rbm_phi_theta(si, theta) * summedgradtimesphi/magnitude**3)
        d += deriv
    return deriv

In [109]:


N,M = 4,0
phi = create_random_state(N)
theta = 0.01*(2*np.random.rand(N + M*(N+1))-1)
print(diff_loss(phi, theta))
print(finite_diff_grad(lambda x: loss(phi, x),theta,1e-6))




[0.06141668-0.02891822j 0.06121141-0.02882157j 0.06180241-0.02909985j
 0.06181539-0.02910596j]
[ 0.09207254 -0.03209053  0.08174556 -0.01313352]


In [110]:
def gradient_descent(phi, M, learning_rate, max_iter = 10000, autoTerminate = False):
    N = int(np.log2(len(phi)))
    theta = 0.01*(2*np.random.rand(N + M*(N+1))-1) + 0.01j*(2*np.random.rand(N + M*(N+1))-1)
    learned_theta = np.copy(theta)
    losses = []
    iteration = 0
    while True:
        grad = diff_loss(phi, learned_theta)
        
        learned_theta -= learning_rate * grad
        losses.append(loss(phi, learned_theta))
        iteration += 1
        if iteration > max_iter:
            break
        if iteration % 100 == 0:
            print(f"Iteration {iteration}, Loss: {losses[-1]}, Loss Change: {losses[-2] - losses[-1]}")
        if autoTerminate and iteration > max_iter/10 and (losses[-2] - losses[-1] < 1e-6 or losses[-1] < 5e-4):
            break

    return theta, learned_theta, losses

In [111]:
N, M = 3, 6
phi= create_random_state(N)
print("Initial random phi:", phi)
print("Gradient descent:")
random_theta, new_theta, _ = gradient_descent(phi, M, 0.01,1000)
print(len(random_theta))
print("Initial loss:", loss(phi, random_theta))
print("Initial random theta:", random_theta)
print("Final loss:", loss(phi, new_theta))
print("Final theta:", new_theta)


Initial random phi: [-0.32225337-0.09298033j  0.03635191-0.32572989j  0.30971452+0.22370556j
  0.23483432+0.12218615j -0.02457195+0.32230655j -0.23392743-0.29923716j
 -0.28537576-0.17929084j  0.31757956-0.31756986j]
Gradient descent:
Iteration 100, Loss: 0.9662736946873922, Loss Change: 0.0001911779456500451
Iteration 200, Loss: 0.9488103163764817, Loss Change: 0.00015935931774413437
Iteration 300, Loss: 0.9342893732428054, Loss Change: 0.00013206467219140094
Iteration 400, Loss: 0.9222361489765332, Loss Change: 0.00011034443986079889
Iteration 500, Loss: 0.9119898531711389, Loss Change: 9.600875397175468e-05
Iteration 600, Loss: 0.9027986538111337, Loss Change: 8.902496907348123e-05
Iteration 700, Loss: 0.8939848767746937, Loss Change: 8.81540296016059e-05
Iteration 800, Loss: 0.8850118472357977, Loss Change: 9.19205895838493e-05
Iteration 900, Loss: 0.8754915824368377, Loss Change: 9.877944930025695e-05
Iteration 1000, Loss: 0.8652228760987555, Loss Change: 0.00010632037119384652
27


In [112]:
import matplotlib.pyplot as plt

def plot_phi_theta(phi, theta, label="phi_theta"):
    N = int(np.log2(len(phi)))
    x = np.linspace(0, N-1, N)
    general_state = all_states(N)
    length = len(general_state)
    magnitude = np.linalg.norm([rbm_phi_theta(general_state[j], theta) for j in range(length)])
    # for real (fr)
    y = [rbm_phi_theta(general_state[i], theta).real/magnitude for i in range(length)]
    plt.plot(x, y, label=label+"real")
    y = [phi[i].real for i in range(length)]
    plt.plot(x, y, label="phi real")
    plt.legend()
    plt.show()
    y = [rbm_phi_theta(general_state[i], theta).imag/magnitude for i in range(length)]
    plt.plot(x, y, label=label+"imag")
    y = [phi[i].imag for i in range(length)]
    plt.plot(x, y, label="phi imag")
    plt.legend()
    plt.show()


In [113]:
# plot_phi_theta(phi, random_theta, label = "Initial random theta")
# plot_phi_theta(phi, new_theta, label = "Final theta")



In [ ]:
N, M = 4, 2
phi= create_random_state(N)
l = 1
while l > 0.001:
    random_theta, new_theta, losses = gradient_descent(phi, M, 0.1, 10000, autoTerminate=True)
    l = loss(phi, new_theta)
    print(f"Final loss with M = {M}:", l)
    M += 1
    plt.plot(losses)
    plt.yscale("log")
    plt.show()
print("The ratio M/N needed to get a loss below 0.001 is", M/N)
plot_phi_theta(phi, random_theta, label = "Initial random theta")
plot_phi_theta(phi, new_theta, label = "Final theta")

Iteration 100, Loss: 0.8957577946832579, Loss Change: 0.000874211722704854
Iteration 200, Loss: 0.837058884763693, Loss Change: 0.00039569801920069647
Iteration 300, Loss: 0.8074599099239734, Loss Change: 0.000222364851213408
Iteration 400, Loss: 0.789763148979966, Loss Change: 0.0001413023988715567
Iteration 500, Loss: 0.778066910693474, Loss Change: 9.706563282030345e-05
Iteration 600, Loss: 0.7698121920861536, Loss Change: 7.034889267842903e-05
Iteration 700, Loss: 0.7637105658663863, Loss Change: 5.30161118064143e-05
Iteration 800, Loss: 0.7590430041364601, Loss Change: 4.115656432313042e-05
Iteration 900, Loss: 0.7553768628864088, Loss Change: 3.270116896991748e-05
Iteration 1000, Loss: 0.7524363920918165, Loss Change: 2.64720884635139e-05
Iteration 1100, Loss: 0.750037659812247, Loss Change: 2.1758983839825774e-05
Iteration 1200, Loss: 0.7480533690342104, Loss Change: 1.8112958990901262e-05
Iteration 1300, Loss: 0.7463926880235376, Loss Change: 1.5239159004476477e-05
Iteration 14